## Flowtron

#### Import libraries and setup matplotlib

In [65]:
###############################################################################
#
#  Copyright (c) 2020, NVIDIA CORPORATION. All rights reserved.
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
###############################################################################
import matplotlib
matplotlib.use("Agg")
import matplotlib.pylab as plt
import IPython.display as ipd

import os
import argparse
import json
import sys
import numpy as np
import torch


from flowtron import Flowtron
from torch.utils.data import DataLoader
from data import Data
from train import update_params

sys.path.insert(0, "tacotron2")
sys.path.insert(0, "tacotron2/waveglow")
from glow import WaveGlow
from scipy.io.wavfile import write

#### Define inference function

In [57]:
def infer(flowtron_path, waveglow_path, output_dir, text, speaker_id, n_frames,
          sigma, gate_threshold, seed, args_config):
    
    # Parse configs.  Globals nicer in this case
    with open(args_config) as f:
        data = f.read()

    global config
    config = json.loads(data)
    update_params(config, args_params)

    data_config = config["data_config"]
    global model_config
    model_config = config["model_config"]
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # load waveglow
    waveglow = torch.load(waveglow_path)['model'].cuda().eval()
    waveglow.cuda().half()
    for k in waveglow.convinv:
        k.float()
    waveglow.eval()

    # load flowtron
    model = Flowtron(**model_config).cuda()
    checkpoint = torch.load(flowtron_path, map_location='cpu')
    if 'model' in checkpoint:
    	state_dict = checkpoint['model'].state_dict()
    else:
    	state_dict = checkpoint['state_dict']
    model.load_state_dict(state_dict)
    model.eval()
    print("Loaded checkpoint '{}')" .format(flowtron_path))

    ignore_keys = ['training_files', 'validation_files']
    trainset = Data(
        data_config['training_files'],
        **dict((k, v) for k, v in data_config.items() if k not in ignore_keys))
    speaker_vecs = trainset.get_speaker_id(speaker_id).cuda()
    text = trainset.get_text(text).cuda()
    speaker_vecs = speaker_vecs[None]
    text = text[None]

    with torch.no_grad():
        residual = torch.cuda.FloatTensor(1, 80, n_frames).normal_() * sigma
        mels, attentions = model.infer(
            residual, speaker_vecs, text, gate_threshold=gate_threshold)

    for k in range(len(attentions)):
        attention = torch.cat(attentions[k]).cpu().numpy()
        fig, axes = plt.subplots(1, 2, figsize=(16, 4))
        axes[0].imshow(mels[0].cpu().numpy(), origin='lower', aspect='auto')
        axes[1].imshow(attention[:, 0].transpose(), origin='lower', aspect='auto')
        fig.savefig(os.path.join(output_dir, 'sid{}_sigma{}_attnlayer{}.png'.format(speaker_id, sigma, k)))
        plt.close("all")

    with torch.no_grad():
        audio = waveglow.infer(mels.half(), sigma=0.8).float()
        saved_audio = waveglow.infer(mels.half(), sigma=0.8)

    saved_audio = saved_audio[0].data.cpu().numpy()
    saved_audio = np.asarray(saved_audio, dtype=np.float32)
        
    audio = audio.cpu().numpy()[0]
    # normalize audio for now
    audio = audio / np.abs(audio).max()
    print(audio.shape)

    write(os.path.join(output_dir, 'sid{}_sigma{}.wav'.format(speaker_id, sigma)),
          data_config['sampling_rate'], 
          audio)
    
    write(os.path.join(output_dir, 'sid{}_sigma{}_FIX.wav'.format(speaker_id, sigma)),
          data_config['sampling_rate'], 
          saved_audio)
    
    return saved_audio

#### Load Inference

In [99]:
'''
if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--config', type=str,
                        help='JSON file for configuration')
    parser.add_argument('-p', '--params', nargs='+', default=[])
    parser.add_argument('-f', '--flowtron_path',
                        help='Path to flowtron state dict', type=str)
    parser.add_argument('-w', '--waveglow_path',
                        help='Path to waveglow state dict', type=str)
    parser.add_argument('-t', '--text', help='Text to synthesize', type=str)
    parser.add_argument('-i', '--id', help='Speaker id', type=int)
    parser.add_argument('-n', '--n_frames', help='Number of frames',
                        default=400, type=int)
    parser.add_argument('-o', "--output_dir", default="results/inferences")
    parser.add_argument("-s", "--sigma", default=0.5, type=float)
    parser.add_argument("-g", "--gate", default=0.5, type=float)
    parser.add_argument("--seed", default=1234, type=int)
    args = parser.parse_args()

    python inference.py -c config.json -f '/home/serkhane/Repositories/AI/models/tts/model_3000' -w '/home/serkhane/Repositories/AI/models/tts/waveglow_256channels_universal_v5.pt' -t "My name is Karim" -i 0
'''

#args_config = 'config.json'
args_config = '/home/serkhane/Repositories/AI/modules/tts/flowtron/config_default.json'
args_params = []
args_output_dir = "results/inferences"
#args_flowtron_path = '/home/ks1/models/tts/flowtron/exp_1_3_video_jacko/outdir/model_20000'
args_flowtron_path = '/home/serkhane/models/tts/flowtron/flowtron_ljs.pt'
args_waveglow_path = '/home/serkhane/models/tts/waveglow/waveglow_256channels_universal_v5.pt'
args_text = "Hello Mister Ryan Cherifa. I'm not a scam, give me your bank card number to add money in your account"
args_id = 0
args_n_frames = 600
args_sigma = 0
args_gate = 0.5
args_seed = 1234

# Parse configs.  Globals nicer in this case
with open(args_config) as f:
    data = f.read()

global config
config = json.loads(data)
update_params(config, args_params)

data_config = config["data_config"]
global model_config
model_config = config["model_config"]

# Make directory if it doesn't exist
if not os.path.isdir(args_output_dir):
    os.makedirs(args_output_dir)
    os.chmod(args_output_dir, 0o775)

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
audio = infer(args_flowtron_path, args_waveglow_path, args_output_dir, args_text, 
              args_id, args_n_frames, args_sigma, args_gate, args_seed, args_config)

Loaded checkpoint '/home/serkhane/models/tts/flowtron/flowtron_ljs.pt')
Number of speakers : 1
Number of speakers : 1
Hitting gate limit
(153344,)


#### Inference

In [100]:
ipd.Audio(audio, rate=data_config['sampling_rate'])